### General flow:
* Each city, state pair is tied to a ZIP code
* Each ZIP code is tied to a coordinate pair of latitude and longitude
* Use the latitude longitude pair to look up the ZIP code
* Use the ZIP code to look up the city, state pair (or most common one among results)
* Add city, state pair to popup

In [ ]:
import pandas as pd
import pickle

pd.set_option("display.max_columns",None) 

In [ ]:
# load the ZIP code to lat/long data
with open('zip_coord_api.pkl','rb') as handle:
    zip_coor = pickle.load(handle)
    
# load the main dataframe
cars = pd.read_pickle('..\cars.pkl')

In [ ]:
# Recast data
cars['mileage'] = cars['mileage'].astype('float')
cars['year'] = cars['year'].astype('float')

# Clean price data and recast
ugly_cars = cars[cars['price'].str.contains('MSRP')]
ugly_cars.index
for index, car in ugly_cars.iterrows():
    if '|' not in car['price']:
        cars.at[index,'price'] = None
    else:
        cars.at[index,'price'] = car['price'].split('|')[0].strip()
cars['price'] = cars['price'].astype('float')

In [ ]:
# Add in a city, state column
cars['citystate_abb'] = cars[['city', 'state']].apply(lambda x: ', '.join(x.astype(str)), axis=1)

# only get non-null expected price rows
good_cars = cars[cars.expected_price.notnull()]

In [ ]:
# zip_coor[84737]
zip_coor

In [ ]:
# reverse the ZIP:lat/long dictionary
coor_zip = {str(round(v[0], 3))+str(round(v[1], 3)): k for k, v in zip_coor.items()}

if len(coor_zip) != len(zip_coor):
    raise ValueError('the reversed ZIP code dictionary had a different number of keys than the original')

# example of how to reverse a dictionary
# inv_map = {v: k for k, v in my_map.items()}

In [ ]:
coor_zip

In [ ]:
# cars
subdf = good_cars.loc[cars['zip_code'] == 84321]
subdf

In [ ]:
# find most common city, state in subdf
subdf.citystate_abb.mode()[0]

In [ ]:
# use mouseover and mouseout events in CircleMarker class

# grab a few locations and associated data
loc0 = good_cars.loc[cars['city'] == 'Salt Lake City'].zip_code.mode()[0]
loc1 = good_cars.loc[cars['city'] == 'Sandy'].zip_code.mode()[0]
loc2 = good_cars.loc[cars['city'] == 'West Jordan'].zip_code.mode()[0]
print(f'coordinates for {loc0} are {zip_coor[loc0]}')
print(f'coordinates for {loc1} are {zip_coor[loc1]}')
print(f'coordinates for {loc2} are {zip_coor[loc2]}')
subdf0 = cars.loc[cars['zip_code'] == loc0]
subdf1 = cars.loc[cars['zip_code'] == loc1]
subdf2 = cars.loc[cars['zip_code'] == loc2]

from ipyleaflet import Map, basemaps, GeoJSON, Popup, FullScreenControl, CircleMarker, MarkerCluster
from traitlets import Instance, Dict, List, Int, Unicode
from ipywidgets import CallbackDispatcher, HTML

# center = [37.0965, -113.5684] # St. George, UT
center = [40.7608, -111.8910] # SLC, UT
# loc0 = center
# loc1 = [center[0]-0.15, center[1]]
# loc2 = [center[0], center[1]+0.15]
zoom = 8
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)
m.add_control(FullScreenControl())

tmpdata = 'test string'

# def bind_data(**kwargs):
# #     print('inside set_data')
# #     print(kwargs)
#     return kwargs['data']

def hover_on(event, type, coordinates):
    
    ################################
    # new strategy: use coordinates to reverse look up data associated with those coordinates
    ################################
    currzip = coor_zip[str(round(coordinates[0], 3))+str(round(coordinates[1], 3))]
    currdata = cars.loc[cars['zip_code'] == currzip]
    currlabel = currdata.citystate_abb.mode()[0]
#     print(currlabel) # this should be the city, state label
#     currtotal = str(currdata.shape[0])
    currtotal = "{:,}".format(currdata.shape[0])
    currmedprice = "{:,}".format(int(currdata.price.median()))
    currmedmile = "{:,}".format(int(currdata.mileage.median()))
    currmedyear = str(int(currdata.year.median()))
    currcommcar = str(currdata.make.mode()[0]) + ' ' + str(currdata.model.mode()[0])
    
    # remove old popup layer
    if isinstance(m.layers[-1], Popup):
        m.remove_layer(m.layers[-1])
    
    # get center of current polygon
#     txt_loc = polylabel(feature['coordinates'])[::-1]

    # add a popup layer on hover over a city
    message = HTML()
# #     message.value = currlabel + '<br>Median Price:  $' + currmedprice + '<br>Median Mileage:  ' + currmedmile

    message.value = ('<h4><strong>' + currlabel + '</strong></h4>' +
                     '<table>' +
                         '<tr><td>Total Cars:&emsp;</td><td>' + currtotal + '</td></tr>' +
                         '<tr><td>Median Price:&emsp;</td><td>$' + currmedprice + '</td></tr>' +
                         '<tr><td>Median Mileage:&emsp;</td><td>' + currmedmile + '</td></tr>' +
                         '<tr><td>Median Year:&emsp;</td><td>' + currmedyear + '</td></tr>' +
                         '<tr><td>Most Common Car:&emsp;</td><td>' + currcommcar + '</td></tr>' +
                     '</table>')
    
#     message.value = ('<h4><strong>Test Title</strong></h4>' + 
# '<table><tr><th>Company</th><th>Contact</th><th>Country</th></tr><tr><td>Alfreds Futterkiste</td><td>Maria Anders</td><td>Germany</td></tr><tr><td>Centro comercial Moctezuma</td><td>Francisco Chang</td><td>Mexico</td></tr></table>')
    
    popup = Popup(location=coordinates, child=message, close_button=False, auto_close=True, close_on_escape_key=False)

    m.add_layer(popup) # add the new layer
    
# def hover_off(**kwargs):
#     print(kwargs)
#     print('inside hover_off')

circobj0 = CircleMarker(location = zip_coor[loc0])
# circobj0 = CircleMarker(location = [round(coor, 3) for coor in zip_coor[loc0]], radius = 20)
circobj0.on_mouseover(hover_on)
# circobj0.on_mouseout(hover_off)

circobj1 = CircleMarker(location = zip_coor[loc1])
circobj1.on_mouseover(hover_on)
# circobj1.on_mouseout(hover_off)

circobj2 = CircleMarker(location = zip_coor[loc2])
circobj2.on_mouseover(hover_on)
# circobj2.on_mouseout(hover_off)

# circobj.data = set_data(data=tmpdata)

m.add_layer(circobj0)
m.add_layer(circobj1)
m.add_layer(circobj2)
m

In [ ]:
# Try out addding a few circles as a marker cluster
from ipyleaflet import LayersControl

center = [37.0965, -113.5684] # St. George, UT
zoom = 9
m2 = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)
m2.add_control(FullScreenControl())

circ1 = CircleMarker(location = center)
circ2 = CircleMarker(location = [pt+0.1 for pt in center])
circ3 = CircleMarker(location = [center[0], center[1]-0.15], radius = 20)
circ4 = CircleMarker(location = [center[0]+0.15, center[1]-0.15], radius = 15)

cluster1 = MarkerCluster(markers=(circ1, circ2, circ3, circ4))

m2.add_layer(cluster1) # cool! this groups them until you zoom in far enough to properly distinguish

# show the map
m2

## Deprecated code below this attempting to append new methods to CircleMarker class

In [ ]:
# see if you can add in an on_hover method to the imported CircleMarker class

from ipyleaflet import Map, basemaps, GeoJSON, Popup, FullScreenControl, CircleMarker, MarkerCluster
from traitlets import Instance, Dict, List, Int, Unicode
from ipywidgets import CallbackDispatcher

center = [37.0965, -113.5684] # St. George, UT
zoom = 9
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)
m.add_control(FullScreenControl())

# def tmpfunc(event, feature):
# #     print('inside tmpfunc!')
#     print(event)
    
#     # remove old popup layer
#     if isinstance(m.layers[-1], Popup):
#         m.remove_layer(m.layers[-1])
    
#     # get center of current polygon
#     txt_loc = polylabel(feature['coordinates'])[::-1]

#     # add a popup layer on hover over a city
#     message = HTML()
#     message.value = feature['varinput']
#     popup = Popup(location=txt_loc, child=message, close_button=False, auto_close=True, close_on_escape_key=False)

#     m.add_layer(popup) # add the new layer

def tmpfunc2(event, feature):
    print('inside tmpfunc!')
    print(event)

def _handle_m_msg(self, _, content, buffers):
    if content.get('event', '') == 'click':
        self._click_callbacks(**content)
    if content.get('event', '') == 'mouseover':
        self._hover_callbacks(**content)

def on_hover(self, callback, remove=False):
    print('here!')
    
    '''
    The hover callback takes an unpacked set of keyword arguments.
    '''
    self._hover_callbacks.register_callback(callback, remove=remove)

# CircleMarker._hover_callbacks = Instance(CallbackDispatcher, ())
# CircleMarker._handle_m_msg = _handle_m_msg
# CircleMarker.on_hover = on_hover

circobj = CircleMarker(location = center)
circobj.on_mouseover(tmpfunc2)

# circobj.on_hover(tmpfunc2)
# circobj.on_hover

m.add_layer(circobj)
m
# dir(circobj)

## Deprecated code below this attempting to inherit methods from other class(es)

In [ ]:
# see if you can add in an on_hover method to the imported CircleMarker class by creating a new class that inherits it

# Make a new class that inherits from CircleMarker
from ipyleaflet import Map, basemaps, GeoJSON, Popup, FullScreenControl, CircleMarker, MarkerCluster
from traitlets import Instance, Dict, List, Int, Unicode
from ipywidgets import CallbackDispatcher
import os, binascii
from ipykernel.comm import Comm

class HoverMap(Map):
    def add_layer(self, layer):
        print('got here XXXXXXXXXXXXXX')
#         if isinstance(layer, dict):
#             layer = basemap_to_tiles(layer)
#         if layer.model_id in self._layer_ids:
#             raise LayerException('layer already in layergroup: %r' % layer)
        self.layers = tuple([l for l in self.layers] + [layer])

center = [37.0965, -113.5684] # St. George, UT
zoom = 9
m = HoverMap(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)
m.add_control(FullScreenControl())

def_loc = [0.0, 0.0]

class HoverCircle(CircleMarker, GeoJSON):
    _view_name = Unicode('LeafletCircleMarkerView').tag(sync=True)
    _model_name = Unicode('LeafletCircleMarkerModel').tag(sync=True)
    
    _hover_callbacks = Instance(CallbackDispatcher, ())
    
    comm = Instance('ipykernel.comm.Comm', allow_none=True)
    
    location = List(def_loc).tag(sync=True)
    
    radius = Int(10, help='radius of circle in pixels').tag(sync=True, o=True)
    
    def __init__(self, **kwargs):
#         def model_id(*argvs):
#             return binascii.b2a_hex(os.urandom(16)).decode('utf-8')
#         self.model_id = model_id()
        
#         print(binascii.b2a_hex(os.urandom(16)).decode('utf-8'))
        
#         self = self._replace(model_id=binascii.b2a_hex(os.urandom(16)).decode('utf-8'))
#         self.model_id = binascii.b2a_hex(os.urandom(16)).decode('utf-8')
#         print(self.model_id)
        pass
        
#         if kwargs.get('data'):
#             print('data found!')
#             self.data = kwargs['data']
#         if kwargs.get('location'):
#             self.location = kwargs['location']
        
    def on_hover(self, callback, remove=False):
        '''
        The hover callback takes an unpacked set of keyword arguments.
        '''
        self._hover_callbacks.register_callback(callback, remove=remove)
        
def tmpfunc(event, feature):
    print('made it inside tmpfunc')
# #     print(event)
    
#     # remove old popup layer
#     if isinstance(m.layers[-1], Popup):
#         m.remove_layer(m.layers[-1])
    
#     # get center of current polygon
#     txt_loc = polylabel(feature['coordinates'])[::-1]

#     # add a popup layer on hover over a city
#     message = HTML()
#     message.value = feature['varinput']
#     popup = Popup(location=txt_loc, child=message, close_button=False, auto_close=True, close_on_escape_key=False)

#     m.add_layer(popup) # add the new layer    

circobj = HoverCircle(data = 5, location = center)
print(f'type of HoverCircle obj is {type(circobj)}')

# print(circobj.model_id)

circobj.on_hover(tmpfunc)

circorig = CircleMarker(location=center)
print(f'type of CircleMarker obj is {type(circorig)}')
circorig2 = CircleMarker(location=center)
print(type(circorig.model_id))
print(circorig2.model_id)

# m.add_layer(circobj)
# m